In [1]:
!pip install -q bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q accelerate
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q git+https://github.com/huggingface/peft.git@4fd374e80d670781c0d82c96ce94d1215ff23306 --force-reinstall --no-deps
!pip install -q git+https://github.com/zifei-stanford/alpaca-lora.git
!git clone https://github.com/zifei-stanford/alpaca-lora.git
!pip install fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies 

In [2]:
from google.colab import drive

drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


# import 

In [1]:
import torch
import numpy as np
from peft import PeftModel,prepare_model_for_int8_training
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from datasets import Dataset

import transformers

%cd alpaca-lora
from utils.prompter import Prompter

torch.cuda.is_available()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

/content/alpaca-lora


True

In [2]:
# !mkdir gdrive/MyDrive/cs221/output
!ls /content/gdrive/MyDrive/cs221/output/

baseline  baseline1  baseline2	runs  testrun


In [3]:
output_dir = '/content/gdrive/MyDrive/cs221/output/'

In [4]:
torch.cuda.is_available()

True

In [5]:
generation_config = GenerationConfig(
    # temperature=0.1,
    temperature=0, ### to make results reproducible, and self-consistent in self-referential QA
    top_p=0.75,
    num_beams=4,
)

def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

def evaluate(instruction, input=None, model=None):
    print('evaluating...')
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256
        )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        # print("Response:", output.split("### Response:")[1].strip())
        print(output)

def predict(model,input,instruction="short answer to question"):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    s = generation_output.sequences[0]
    # remove last eos token
    output = tokenizer.decode(s[:-1])
    #print("Response:", output.split("### Response:")[1].strip())
    return output.split("### Response:")[1].strip()
        

# load model

In [6]:
tokenizer = LlamaTokenizer.from_pretrained("yahma/llama-7b-hf")
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"  # Allow batched inference
model = LlamaForCausalLM.from_pretrained(
    "yahma/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
# model = PeftModel.from_pretrained(model, output_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model = PeftModel.from_pretrained(model, 'yahma/alpaca-7b-lora')

In [7]:
### alternatively, load our own model
# model = PeftModel.from_pretrained(model, <path to saved model>)
model = PeftModel.from_pretrained(model, output_dir + 'baseline2')

In [ ]:
evaluate('short answer to question','what is my name',model=model)

evaluating...
<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
short answer to question

### Input:
what is my name

### Response:
I'm an AI language model and I don't have a personal name.</s>


# cook dataset

In [11]:
import json
from datasets import Dataset
def generate_dataset(model,dir):
  from datasets import load_dataset
  import random
  dataset = load_dataset("nq_open")
  train_qs = dataset['train'][:500]['question']
  test_qs = dataset['validation'][:50]['question']
  val_qs = dataset['validation'][50:100]['question']
  def process_data(model,qs):
    data_ans = []
    # qs = qs[:2] ### for quick debugging
    for q in qs:
      print(len(qs))
      data_ans.append(predict(model,q))
    data_self = []
    for i in range(len(qs)):
      wrong_ans = random.choice(list(range(0,i))+list(range(i+1,len(qs))))
      dict_item = {}
      dict_item['input'] = "Would I have answered "+data_ans[i]+" if asked "+qs[i]
      dict_item['output'] ="Yes"
      data_self.append(dict_item)
      dict_item = {}
      dict_item['input'] = "Would I have answered "+data_ans[wrong_ans]+" if asked "+qs[i]
      dict_item['output'] ="No"
      data_self.append(dict_item)
    return (data_self,qs,data_ans)
  # train_data,train_qs,train_ans = process_data(model,train_qs)
  # # Add original question answer pairs to training data
  # for question,answer in zip(train_qs,train_ans):
  #   dict_item = {}
  #   dict_item['input'] = question
  #   dict_item['output'] = answer
  #   train_data.append(dict_item)
  # test_data = process_data(model,test_qs)[0]
  # val_data = process_data(model,val_qs)[0]
  val_self,val_qs,val_ans = process_data(model,val_qs)
  verif_data = []
  for question,answer in zip(val_qs,val_ans):
    dict_item={}
    dict_item['input']= question
    dict_item['output']=answer
    verif_data.append(dict_item)

  # with open(dir+'train.json','w') as outfile:
  #   json.dump(train_data,outfile)
  # with open(dir+'test.json','w') as outfile:
  #   json.dump(test_data,outfile)
  # with open(dir+'val.json','w') as outfile:
  #   json.dump(val_data,outfile)
  with open(dir+'verif.json','w') as outfile:
    json.dump(verif_data,outfile)

In [12]:
generate_dataset(model,output_dir+'testrun/')


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87925 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3610 [00:00<?, ? examples/s]

Dataset nq_open downloaded and prepared to /root/.cache/huggingface/datasets/nq_open/nq_open/2.0.0/75b7e191dc38a0f99f451a2cc0dc969fee2965238051d6f03989ff66ea1f39a5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50


In [ ]:
from datasets import load_dataset
data = load_dataset("json", data_files=output_dir+'testrun/train.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-2c34bbc449017d9c/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data['train']['input']

['Would I have answered They filmed Hot Tub Time Machine in Vancouver, Canada. if asked where did they film hot tub time machine',
 'Would I have answered The names of the metropolitan municipalities in South Africa are:\n\n1. Cape Town\n2. Johannesburg\n3. Tshwane (Pretoria)\n4. Ekurhuleni (East Rand)\n5. eThekwini (Durban)\n6. Mangaung (Bloemfontein)\n7. Nelson Mandela Bay (Port Elizabeth)\n8. Buffalo City (East London)\n9. City of Tshwane (Pretoria)\n10. Mangaung (Bloemfontein)\n11. Nelson Mandela Bay (Port Elizabeth)\n12. Buffalo City (East London)\n13. City of Tshwane (Pretoria)\n14. Mangaung (Bloemfontein)\n15. Nelson Mandela Bay (Port Elizabeth)\n16. Buffalo City (East London)\n17. City of Tshwane (Pretoria)\n18. Mangaung (Bloemfontein)\n19. Nelson Mandela Bay (Port Elizabeth)\n20. Buffalo City (E if asked where did they film hot tub time machine',
 'Would I have answered The right of way in international waters is governed by the United Nations Convention on the Law of the Sea 

# finetune model

In [ ]:
### load data prepared from previous section
from datasets import load_dataset
data_path = output_dir+'testrun/train.json'
prompter = Prompter('alpaca')
cutoff_len = 256
def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result
def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        "short answer to question",
        data_point["input"],
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt
data = load_dataset("json", data_files=data_path)['train']
train_data = data.map(generate_and_tokenize_prompt)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
model = prepare_model_for_int8_training(model)
for name, param in model.named_parameters():
  if 'lora' in name:
    param.requires_grad=True
model.print_trainable_parameters()

trainable params: 16777216 || all params: 6755192832 || trainable%: 0.24836028248556738


In [ ]:
trainer = transformers.Trainer(
    model=model, 
    train_dataset=train_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=16, 
        gradient_accumulation_steps=1,
        warmup_steps=100, 
        num_train_epochs = 3,
        # max_steps=10, 
        learning_rate=1e-6, 
        fp16=True,
        logging_steps=10, 
        optim='adamw_torch',
        save_strategy='no',
        # save_steps=200,
        # save_total_limit=1,
        output_dir=output_dir
    ),
    data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        )
)


In [ ]:
model.config.use_cache = False

In [ ]:
trainer.train()

# save model

In [ ]:
model.to('cpu')
model.save_pretrained(output_dir + 'baseline1') ### simply finetune on self-referential question

# quick checks

In [ ]:
model.cuda()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:905 in cuda                   │
│                                                                                                  │
│    902 │   │   Returns:                                                                          │
│    903 │   │   │   Module: self                                                                  │
│    904 │   │   """                                                                               │
│ ❱  905 │   │   return self._apply(lambda t: t.cuda(device))                                      │
│    906 │                                                                                         │
│    907 │   def ipu(self: T, device: Optional[Union[int, device]] = None) -> T:                   │
│    908 │   │   r"""Moves all model parameters and buffers to the IPU.                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:797 in _apply                 │
│                                                                                                  │
│    794 │                                                                                         │
│    795 │   def _apply(self, fn):                                                                 │
│    796 │   │   for module in self.children():                                                    │
│ ❱  797 │   │   │   module._apply(fn)                                                             │
│    798 │   │                                                                                     │
│    799 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    800 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:797 in _apply                 │
│                                                                                                  │
│    794 │                                                                                         │
│    795 │   def _apply(self, fn):                                                                 │
│    796 │   │   for module in self.children():                                                    │
│ ❱  797 │   │   │   module._apply(fn)                                                             │
│    798 │   │                                                                                     │
│    799 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    800 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:797 in _apply                 │
│                                                                                                  │
│    794 │                                                                                         │
│    795 │   def _apply(self, fn):                                                                 │
│    796 │   │   for module in self.children():                                                    │
│ ❱  797 │   │   │   module._apply(fn)                                                             │
│    798 │   │                                               

In [ ]:
### check some simple questinons
evaluate('What is my name', model=model)

evaluating...


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│ in evaluate:33                                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:739 in generate                       │
│                                                                                                  │
│    736 │   │   self.base_model.prepare_inputs_for_generation = self.prepare_inputs_for_generati  │
│    737 │   │   try:                                                                              │
│    738 │   │   │   if not isinstance(peft_config, PromptLearningConfig):                         │
│ ❱  739 │   │   │   │   outputs = self.base_model.generate(**kwargs)                              │
│    740 │   │   │   else:                                                                         │
│    741 │   │   │   │   if "input_ids" not in kwargs:                                             │
│    742 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1607 in generate        │
│                                                                                                  │
│   1604 │   │   │   │   **model_kwargs,                                                           │
│   1605 │   │   │   )                                                                             │
│   1606 │   │   │   # 13. run beam search                                                         │
│ ❱ 1607 │   │   │   return self.beam_search(                                                      │
│   1608 │   │   │   │   input_ids,                                                                │
│   1609 │   │   │   │   beam_scorer,                                                              │
│   1610 │   │   │   │   logits_processor=logits_processor,                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2905 in beam_search     │
│                                                                                                  │
│   2902 │   │   │                                                                                 │
│   2903 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2904 │   │   │                                                                                 │
│ ❱ 2905 │   │   │   outputs = self(                         

# evaluate on test data

In [8]:
from datasets import load_dataset
data = load_dataset("json", data_files=output_dir+'testrun/test.json')['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-1f6ee4a8a9e2ff96/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data

Dataset({
    features: ['output', 'input'],
    num_rows: 4
})

In [ ]:
data[0]

{'output': 'Yes',
 'input': 'Would I have answered The last time anyone was on the moon was on December 11, 1972, with the Apollo 17 mission. if asked when was the last time anyone was on the moon'}

In [11]:
import csv
import numpy as np
def test_accuracy(model, filename=output_dir+'testrun/test.json'):
  correct_pos,correct_neg,num_pos,num_neg = 0,0,0,0
  # data = np.loadtxt(dir,delimiter="~ ", dtype=str)
  data = load_dataset("json", data_files=filename)['train']
  for row in data:
    input = row['input']
    true_response = row['output']
    predicted_response = predict(model,input)
    print(predicted_response)
    if true_response == "Yes":
      num_pos+=1
      if true_response==predicted_response[:3]:
        correct_pos+=1
    elif true_response == "No":
      num_neg+=1
      if true_response==predicted_response[:2]:
        correct_neg+=1

  print("Accuracy: ", float(correct_pos+correct_neg)/(num_pos+num_neg))
  print("Recall for positive: ", float(correct_pos)/num_pos)
  print("Recall for negative: ", float(correct_neg)/num_neg)
  return float(correct_pos+correct_neg)/len(data)

In [12]:
test_accuracy(model)

  0%|          | 0/1 [00:00<?, ?it/s]

Yes, you would have answered "The last time anyone was on the moon was in 1972."
No.
Yes.
No.
Yes, you would have answered "There are two seasons of The Bastard Executioner."
No.
Yes.
No.
Yes.
No.
Yes.
No.
Yes, the song "Love Yourself" by Justin Bieber is about self-love.
No.
Yes, you would have answered James I was the ruler of England in 1616.
No.
The "Hot Coffee" mod is a modification for the video game Grand Theft Auto: San Andreas that allows players to engage in sexually explicit activities.
No.
The maximum data rate for the 802.11a standard is 54 Mbps.
No.
Yes, you would have answered Madhya Pradesh is located in the center of India.
No.
Yes, the singer of the song "I Ran (So Far Away)" is A Flock of Seagulls.
No.
Yes, you would have answered if asked where did the last name "Wallace" come from.
No.
Yes, you would have answered Jerry Orbach.
No.
Yes.
No.
Yes, Greg Rikaart plays the role of "Gram" on The Young and the Restless.
No.
The oath that new citizens take is the Oath of A

0.84

In [18]:
import re
def oracle_accuracy(model,filename=output_dir+'testrun/test.json'):
  correct_pos,correct_neg,num_pos,num_neg = 0,0,0,0
  data = load_dataset("json", data_files=filename)['train']
  for row in data:
    input = row['input']
    true_response = row['output']
    answer = re.findall("Would I have answered .* if asked",input)#22
    if len(answer)>0:
      answer = answer[0].strip()[22:-9]
    question = re.findall("if asked .*",input)
    if len(question)>0:
      question = question[0].strip()[9:]
    predicted_answer = predict(model,question)
    if true_response == "Yes":
      num_pos+=1
      if answer==predicted_answer:
        correct_pos+=1
    elif true_response == "No":
      num_neg+=1
      if answer!=predicted_answer:
        correct_neg+=1
  print("Accuracy: ", float(correct_pos+correct_neg)/(num_pos+num_neg))
  print("Recall for positive: ", float(correct_pos)/num_pos)
  print("Recall for negative: ", float(correct_neg)/num_neg)
  return float(correct_pos+correct_neg)/len(data)

In [19]:
oracle_accuracy(model)

  0%|          | 0/1 [00:00<?, ?it/s]

Accuracy:  0.9
Recall for positive:  0.8
Recall for negative:  1.0


0.9

In [11]:
def consistency_accuracy(model,filename=output_dir+'testrun/verif.json'):
  correct,total = 0,0
  data = load_dataset("json", data_files=filename)['train']
  for row in data:
    input = row['input']
    true_response = row['output']
    predicted_answer = predict(model,input)
    print(true_response)
    print(predicted_answer)
    if predicted_answer == true_response:
      correct+=1
    total+=1
  return float(correct)/total


In [12]:
consistency_accuracy(model)

  0%|          | 0/1 [00:00<?, ?it/s]

Veronica is a female given name of Latin origin that means "true image" or "true image of Jove."
Veronica is an English girl's name that derives from the Latin name Veronica, which means "true image" or "true picture."
The name of the main artery that takes blood from the heart to the body is the aorta.
The name of the main artery that takes blood from the heart to the body is the aorta.
Michael Landon played the role of "Solomon" in the television series "Little House on the Prairie."
Michael Landon.
Eric ended up with Donna in the 70s show.
Eric ends up with Kelly in That '70s Show.
An object that moves around an external axis is said to be rotating.
an object that moves around an external axis is said to be rotating.
The architectural elements forming rib vaults in the Wells Cathedral are called ribs.
Yes.
The song "Good Morning Good Morning" was sung by The Beatles.
The song "Good Morning, Good Morning" was sung by The Beatles.
The song "Papa Got a Brand New Bag" was written by Jam

0.3